#Capstone Project- The Battle of Neighborhoods

    ###Introduction

Our goal is to find the best location in Thessaloniki, Greece where he can open a vegan restaurant.

Veganism is becoming more and more popular nowadays. The reason for these are that more people are becoming health conscious and concerned about ethical eating. 
There are a lot of non-vegans who love vegan food. Those who genuinely love a healthy lifestyle, for instance. So, a good location to open the restaurant is near yoga and pilates studios.
Also we will make sure that we choose a highly visible location and a location not near other vegan restaurants.

###Data

####1.Geographic locations of vegan restaurants of Thessaloniki

In [1]:
import pandas as pd
import numpy as np

In [10]:
import requests


In [2]:
# import functionality to find and load dotenv credentials
# and getenv to get environment variables from OS
from os import getenv
from dotenv import load_dotenv, find_dotenv

# load environment keys, it will automatically find and load .env
load_dotenv(find_dotenv())

True

In [5]:
'''
def login(api_key, secret):
    print('Logged with:')
    print('API: ' + api_id)
    print('SECRET: ' + secret_id)'''

# Logging in to third party app with environment variable credentials
api_id = getenv('CLIENT_ID')
secret_id = getenv('CLIENT_SECRET')
VERSION = '20180605'
#login(api_id, secret_id)

In [6]:
Thessaloniki=[40.736851, 22.920227]

In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    api_id, 
    secret_id, 
    VERSION, 
    Thessaloniki[0], 
    Thessaloniki[1], 
    radius, 
    LIMIT)
#url # display URL

In [11]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6559a04eefd00025b733d4'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Oraiókastro',
  'headerFullLocation': 'Oraiókastro',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 40.741351004500004,
    'lng': 22.926154827546327},
   'sw': {'lat': 40.7323509955, 'lng': 22.914299172453674}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4fae4470e4b08885ff37b408',
       'name': 'Σχολή Χορού',
       'location': {'lat': 40.73544351951442,
        'lng': 22.91902034961396,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.73544351951442,
          '

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']